# Joint conf-call report #3

In [1]:
import datetime
print(str(datetime.datetime.today()))

2018-05-04 11:53:28.257043


# Iterations over time lower bound comparison
The speed lower bound is $V_l = \frac{\lambda}{1+\sum_{i=1}^{k} \frac{1}{i}}$. The lower bound of the iterations' amount is
$$\#it = V_l * time$$
so we're dealing with a line $f(x) = V_l x$.

For $n=10$ and $\lambda = 1$:

| Graph | degree $k$ | $V_l$
| :--- |:---: | ---:
| Clique |  $9$  | $\sim 0.34$
| Root and diameter expanders | $2$  | $0.4$
| Cycle |  $1$  | $0.5$
| Diagonal* | $0$  | $1$

_(*) NB: $V_l$ for diagonal graph is not reliable._

### Iterations over time LB plot for _10k samples GD test 003_
![Iter/time LB for 10k samples](media/img/tests/test_003_10ksamples_classic/1_iter-lb_time.png)

# Single node model inspection